In [ ]:
import torch
import numpy as np

def fill_triangular(x, upper=False, name=None):
    """Creates a (batch of) triangular matrix from a vector of inputs.

    Args:
        x: `Tensor` representing lower (or upper) triangular elements.
        upper: Python `bool` representing whether output matrix should be upper
          triangular (`True`) or lower triangular (`False`, default).
        name: Python `str`. The name to give this op (not used in PyTorch version).

    Returns:
        tril: `Tensor` with lower (or upper) triangular elements filled from `x`.

    Raises:
        ValueError: if `x` cannot be mapped to a triangular matrix.
    """

    # Get the last dimension size (m)
    m = x.shape[-1]

    # Calculate n from m using the quadratic formula
    if m is not None:
        m = int(m)
        n = int(np.sqrt(0.25 + 2. * m) - 0.5)
        if n != np.floor(n):
            raise ValueError(f"Input right-most shape ({m}) does not correspond to a triangular matrix.")
    else:
        m = x.shape[-1]
        n = int(torch.sqrt(0.25 + 2. * m).item() - 0.5)

    # Get the batch shape of x (excluding the last dimension)
    batch_shape = x.shape[:-1]

    # Create the new shape for the output tensor (batch dimensions, n x n)
    new_shape = batch_shape + (n, n)

    # Reshape x to the new shape
    x = x.view(-1, m)  # Flatten the last dimension to make it 2D for reshaping
    if upper:
        # Upper triangular: first part is x, the second part is the reverse of the last elements
        x_list = [x, torch.flip(x[..., n:], dims=[-1])]
    else:
        # Lower triangular: first part is reversed x, the second part is x
        x_list = [x[..., n:], torch.flip(x, dims=[-1])]

    x = torch.cat(x_list, dim=-1).view(*new_shape)

    # Create a triangular matrix using torch.tril or torch.triu
    if upper:
        x = torch.triu(x, diagonal=0)  # Keep the upper triangular part
    else:
        x = torch.tril(x, diagonal=0)  # Keep the lower triangular part

    return x


In [ ]:
def fill_triangular(x, upper=False):
    """
    从输入向量创建一个 (批量的) 三角矩阵。

    参数:
        x: `Tensor`, 表示下三角 (或上三角) 元素。
        upper: Python `bool`, 表示输出矩阵是否应为上三角矩阵 (`True`) 或下三角矩阵 (`False`, 默认) 。

    返回:
        tril: `Tensor`, 填充了从 `x` 中提取的下三角 (或上三角) 元素的矩阵。

    异常:
        ValueError: 如果 `x` 无法映射到三角矩阵。
    """
    # 将输入转换为 PyTorch 张量
    x = torch.as_tensor(x)

    # 获取输入张量的最后一个维度大小 (m) 
    m = x.shape[-1]

    # 通过二次公式计算矩阵的维度 n
    # 公式 : n * (n + 1) / 2 = m
    n = int((math.sqrt(8 * m + 1) - 1) / 2)
    if n * (n + 1) // 2 != m:
        raise ValueError(f'输入的最右侧维度大小 ({m}) 无法对应一个三角矩阵。')

    # 初始化输出矩阵, 形状为 [..., n, n], 填充为 0
    output_shape = list(x.shape[:-1]) + [n, n]
    matrix = torch.zeros(output_shape, dtype=x.dtype, device=x.device)

    # 使用三角矩阵的索引填充矩阵
    if upper:
        # 如果是上三角矩阵, 填充上三角部分
        # 使用 torch.triu_indices 生成上三角的索引
        row_indices, col_indices = torch.triu_indices(n, n)
        matrix[..., row_indices, col_indices] = x
    else:
        # 如果是下三角矩阵, 填充下三角部分
        # 使用 torch.tril_indices 生成下三角的索引
        row_indices, col_indices = torch.tril_indices(n, n)
        matrix[..., row_indices, col_indices] = x

    return matrix

def fill_triangular_inverse(x, upper=False):
    """
    将 (批量的) 三角矩阵转换为向量。

    参数 : 
        x: 表示下三角或上三角元素的张量。
        upper: 布尔值, 表示输出矩阵是否为上三角矩阵 (True) 或下三角矩阵 (False, 默认) 。

    返回 : 
        flat_tril: 表示从 x 中向量化的下三角或上三角元素的向量形张量。
    """

    # 获取最后一个维度的大小 (n)
    n = x.shape[-1]
    # 计算向量的长度 (m)
    m = (n * (n + 1)) // 2
    # 获取张量的维度数量
    ndims = len(x.shape)

    if upper:
        # 如果是上三角矩阵
        initial_elements = x[..., 0, :]  # 获取第一行元素
        triangular_portion = x[..., 1:, :]  # 获取剩余的三角部分
    else:
        # 如果是下三角矩阵
        initial_elements = torch.flip(x[..., -1, :], dims=[ndims - 2])  # 获取最后一行并反转
        triangular_portion = x[..., :-1, :]  # 获取剩余的三角部分

    # 反转三角部分并相加
    rotated_triangular_portion = torch.flip(torch.flip(triangular_portion, dims=[ndims - 1]), dims=[ndims - 2])
    consolidated_matrix = triangular_portion + rotated_triangular_portion

    # 重塑三角部分并连接初始元素
    end_sequence = torch.reshape(consolidated_matrix, x.shape[:-2] + (n * (n - 1),))
    y = torch.cat([initial_elements, end_sequence[..., :m - n]], dim=-1)

    return y


In [ ]:
import torch

def make_k_mers(sequences, k, pivot_left=True):
    """
    将 one-hot 编码的核苷酸序列映射到 k-mer 表示。

    参数 : 
        sequences: 形状为 (b, L, 5) 的张量, 表示长度为 L 的序列。
                   假定最后一个维度是 one-hot 编码, 其中 "N" 对应于最后一个位置。
        k: 指定 k-mer 长度的整数。
        pivot_left: 指定是否将 k-mer 旋转到左侧或右侧的布尔值。

    返回 : 
        形状为 (b, L, 4**k-1, 4) 的张量。如果 pivot_left 为 True, 
        则最后一个维度对应于 k-mer 最左侧位置的 4 个可能的核苷酸。
        否则, 最后一个维度对应于 k-mer 最右侧的位置。
        如果 k-mer 包含 N, 则在可能位于该位置的 4 个常规核苷酸中等概率表示。
    """
    L = sequences.shape[-2]
    n = sequences.shape[-1] - 1  # 字母表大小是字符数减 1 (N)
    n = torch.tensor(n, dtype=sequences.dtype)
    # 在 N 的情况下, 在字母表上均匀分布
    sequences_no_N = sequences[..., :-1]
    N_pos = (sequences[..., -1:] == 1).to(sequences.dtype)
    sequences_no_N += (1 / n) * N_pos
    # 计算跨越序列边界的 k-mer 的填充
    pad = torch.ones_like(sequences_no_N[:, :k - 1, :], dtype=sequences.dtype) / n
    if pivot_left:
        sequences_padded_no_N = torch.cat([sequences_no_N, pad], dim=-2)
        k_mers = sequences_padded_no_N[:, :L, None, :]
    else:
        sequences_padded_no_N = torch.cat([pad, sequences_no_N], dim=-2)
        k_mers = sequences_padded_no_N[:, k - 1:L + k - 1, None, :]
    if pivot_left:
        iteration_range = range(1, k)
    else:
        iteration_range = range(k - 2, -1, -1)

    for i in iteration_range:
        shift_i = sequences_padded_no_N[:, i:L + i, None, :, None]
        k_mers = k_mers[..., None, :] * shift_i
        if pivot_left:
            shape = [4**i, 4]
        else:
            shape = [4**(k - i - 1), 4]
        k_mers = k_mers.reshape(list(k_mers.shape[:-3]) + shape)
    return k_mers

def encode_kmer_string(kmer, pivot_left=True, alphabet="ACGT"):
    """
    将 k-mer 转换为格式为 (i, j) 的类, 其中 i < n^{k-1} 且 j < n, 其中 n 是字母表大小。
    例如, AAA -> (0, 0), AAT -> (3, 0), TAA -> (0, 3) 如果 pivot_left 为 True, 否则
        AAA -> (0, 0), AAT -> (0, 3), TAA -> (12, 0)
    输出是 A、C、G、T 情况下的这些类的 one-hot 编码。
    如果 k-mer 包含 N, 则在 4 个常规核苷酸中等概率表示。
    """
    alphabet_with_unknown = alphabet + "N"
    kmer = [alphabet_with_unknown.index(x) for x in kmer]
    kmer = torch.tensor(kmer)
    one_hot = torch.nn.functional.one_hot(kmer, num_classes=len(alphabet_with_unknown)).to(torch.float32) #修正了这一行
    encoded_kmers = make_k_mers(one_hot.unsqueeze(0), k=len(kmer), pivot_left=pivot_left)
    if pivot_left:
        return encoded_kmers.squeeze(0)[0]
    else:
        return encoded_kmers.squeeze(0)[-1]

# 测试用例
sequences = torch.tensor([[[1, 0, 0, 0, 0],
                        [0, 1, 0, 0, 0],
                        [0, 0, 1, 0, 0],
                        [0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 1]]], dtype=torch.float32)
k = 3

k_mers_left = make_k_mers(sequences, k, pivot_left=True)
k_mers_right = make_k_mers(sequences, k, pivot_left=False)

print("k_mers_left shape:", k_mers_left.shape)
print("k_mers_right shape:", k_mers_right.shape)

kmer_string = "ACGN"
encoded_kmer_left = encode_kmer_string(kmer_string, pivot_left=True)
encoded_kmer_right = encode_kmer_string(kmer_string, pivot_left=False)

print("encoded_kmer_left shape:", encoded_kmer_left.shape)
print("encoded_kmer_right shape:", encoded_kmer_right.shape)

print("k_mers_left:", k_mers_left)
print("k_mers_right:", k_mers_right)
print("encoded_kmer_left:", encoded_kmer_left)
print("encoded_kmer_right:", encoded_kmer_right)

In [ ]:
import torch

def inverse_softplus(features):
    """
    计算 softplus 函数的逆函数。

    参数 : 
        features: 输入张量。

    返回 : 
        softplus 函数的逆函数的结果张量。
    """
    # 转换为 float64 以防止大条目的溢出
    features64 = features.double()
    result = torch.log(torch.expm1(features64))
    # 转换回 `features` 的原始数据类型
    return result.to(features.dtype)